# Task 3: Quantum Circuit Simulator

We implement a simple quantum gate simulator for single-qubit and single control gates using python from scratch. The big Indian convention is being used. Then we use the simulator to implementing some simple quantum circuits and finding the optimal parameters of a variational quantum circuit.

In [1]:
import numpy as np
import itertools, random
from cmath import exp
from scipy.optimize import minimize

In [2]:
## Defining the single qubit gates
# identity gate
i = [[1,0],
     [0,1]]
# Pauli-X gate
x = [[0,1],
     [1,0]]
# Pauli-Y gate
y = [[0,-1j],
      [1j,0]]
# Pauli-Z gate
z = [[1,0],
     [0,-1]]
# Hadamard Gate
h = [[1/np.sqrt(2),1/np.sqrt(2)],
     [1/np.sqrt(2),-1/np.sqrt(2)]]
# s gate
s = [[1,0],
     [0,1j]]
# s dagger gate
sdag = [[1,0],
        [0,-1j]]
# T gate
t = [[1,0],
     [0,exp(1j*np.pi/4)]]
# T dagger gate
tdag = [[1,0],
        [0,-exp(1j*np.pi/4)]]

single_gates = {'i':i, 'x':x, 'y':y, 'z':z, 'h':h, 
               's':s, 'sdag':sdag, 't':t, 'tdag':tdag}

In [3]:
# u3 gate is the most general parametric single qubit gate
def u3(theta1,theta2,theta3):
    a11 = np.cos(theta1/2)
    a12 = -exp(1j*theta3)*np.sin(theta1/2)
    a21 = exp(1j*theta2)*np.sin(theta1/2)
    a22 = exp(1j*(theta2+theta3))*np.cos(theta1/2)
    val = [[a11,a12],
           [a21,a22]]
    return val

In [4]:
## Defining some useful tools

def qubit_num(circ):
    # it calculates how many qubits in a quantum circuit
    # returns the number of qubits in the quantum circuit
    # if you want to add some empty qubits in your circuit,  
    # -apply an identity gate to that qubit
    a = list()
    for i in range(len(circ)):
        a.append(max(circ[i]["target"]))
    n = max(a)+1
    return n

def convert_tup(tup):
    # converts a tuble of numbers into a joined string
    str =  ''.join(tup) 
    return str

def bin_permuts(bits_num):
    # returns the various combintions of an n bit string
    lst = list(itertools.product(['0', '1'], repeat=bits_num))
    bin_lst = list()
    for i in range(len(lst)):
        bin_lst.append(convert_tup(lst[i]))
    return bin_lst

def remove_c(s):
    # removes the letter "c" from a given string
    return s.replace('c', '')

In [5]:
## defining main functions of the simulator

def get_ground_state(num_qubits):
    # return vector of size 2**num_qubits with all zeroes except first element which is 1
    vec = np.zeros(2**num_qubits)
    vec[0] = 1
    return vec

def get_single_gate_operator(total_qubits, gate_unitary, target_qubits):
    # return unitary operator of size 2**n x 2**n for single qubit gates
    l = list()
    for qubit in range(total_qubits):
        # apply identity gates for all the qubits
        l.append([[1,0],[0,1]])
    
    # replace target qubits with required unitaries
    l[target_qubits[0]] = single_gates[gate_unitary]

    # tensor product for all qubit unitaries
    o = l[0]
    for qubit in range(1,total_qubits):
        o = np.kron(o,l[qubit])
    return o

def get_double_gate_operator(total_qubits, gate_unitary, target_qubits):
    ## return unitary operator of size 2**n x 2**n for single-qubit controlled gates
    # removing the first letter for a controlled operation to make use of the
    # - already implemented single qubit gates
    unitary = remove_c(gate_unitary)

    P0x0 = [[1, 0],
            [0, 0]]

    P1x1 = [[0, 0],
            [0, 1]]

    lst0 = list()
    lst1 = list()
    
    for qubit in range(total_qubits):
        lst0.append([[1,0],[0,1]])
        lst1.append([[1,0],[0,1]])
        
    lst0[target_qubits[0]] = P0x0
    lst1[target_qubits[0]] = P1x1

    #lst0[target_qubits[1]] = [[1,0],[0,1]]
    lst1[target_qubits[1]] = single_gates[unitary]

    o1 = lst0[0]
    for qubit in range(1,total_qubits):
        o1 = np.kron(o1,lst0[qubit])
        
    o2 = lst1[0]
    for qubit in range(1,total_qubits):
        o2 = np.kron(o2,lst1[qubit])
        
    o = np.array(o1) + np.array(o2)
    return o

def get_parametric_operator(total_qubits, gate_unitary, target_qubits):
    # return a single-qubit parametric gate
    l = list()
    for qubit in range(total_qubits):
        l.append([[1,0],[0,1]])

    l[target_qubits[0]] = gate_unitary

    o = l[0]
    for qubit in range(1,total_qubits):
        o = np.kron(o,l[qubit])
    return o

def get_operator(total_qubits, gate_unitary, target_qubits):
    # return unitary operator of size 2**n x 2**n for given gate and target qubits
    if len(target_qubits)==1:
        o = get_single_gate_operator(total_qubits, gate_unitary, target_qubits)
    elif len(target_qubits)==2:
        o = get_double_gate_operator(total_qubits, gate_unitary, target_qubits)
    else:
        # the simulator is defined for only single and double qubit gates.
        # the following lines return identity gate for multi-qubit operations
        o = [[1,0],[0,1]]
        for qubit in range(1,total_qubits):
            o = np.kron(o,o)
    return o

def run_program(initial_state, program):
    # read program, and for each gate:
    # - calculate matrix operator
    # - multiply state with operator
    # return final state
    psi = initial_state
    for i in range(len(program)):
        gate = program[i]['gate']
        target = program[i]['target']
        
        if gate == 'u3':
            theta1 = program[i]['params']['theta']
            theta2 = program[i]['params']['phi']
            theta3 = program[i]['params']['lambda']
            unitary = u3(theta1, theta2, theta3)
            o = get_parametric_operator(n, unitary, target)
        else:
            o = get_operator(n, gate, target)
        psi = np.dot(psi, o)

    final_state = psi
    return final_state

def measure_all(state_vector):
    # choose element from state_vector using weighted random and return it's index
    states = bin_permuts(n)
    weights = [abs(i)**2 for i in state_vector]
    index = random.choices(states, weights)
    return index[0]

def get_counts(state_vector, num_shots):
    # returns the experimental possible outcomes of the quantum circuit
    states = list()
    for i in range(num_shots):
        states.append(measure_all(state_vector))

    counts = dict()
    for state in states:
        if state not in counts:
            counts[state] = 1
        else:
            counts[state] = counts[state] + 1
    return counts

## Quantum Circuit Testcases

We test our quantum simulator with generic quantum circuits.

### 1: Bell state Quantum circuit

In [6]:
program = [
{ "gate": "h", "target": [0] },
{ "gate": "cx", "target": [0,1] }
]

n = qubit_num(program)

initial_state = get_ground_state(n)

final_state = run_program(initial_state, program)

num_shots = 10000

get_counts(final_state, num_shots)

{'11': 4964, '00': 5036}

### 2: GHZ state (8 qubits)

In [7]:
program = [
{ "gate": "h", "target": [0] },
{ "gate": "cx", "target": [0,1]},
{ "gate": "cx", "target": [1,2]},
{ "gate": "cx", "target": [2,3]},
{ "gate": "cx", "target": [3,4]},
{ "gate": "cx", "target": [4,5]},
{ "gate": "cx", "target": [5,6]},
{ "gate": "cx", "target": [6,7]}
 ]
n = qubit_num(program)

initial_state = get_ground_state(n)

final_state = run_program(initial_state, program)

num_shots = 10000

get_counts(final_state, num_shots)

{'11111111': 5015, '00000000': 4985}

### 3: SWAP gate

In [8]:
program = [
{ "gate": "cx", "target": [0,1]},
{ "gate": "cx", "target": [1,0]},
{ "gate": "cx", "target": [0,1]},
]
n = qubit_num(program)

# initializing the circuit to be in |01> 
# by applying swap gate we expect to get the |10> state
initial_state = [0,1,0,0]

final_state = run_program(initial_state, program)

num_shots = 10000

get_counts(final_state, num_shots)

{'10': 10000}

### 4: 2-qubit Grover's Algorithm 

We implement Grover's Algorithm for the case of two qubits and for marking element $|s\rangle = |11\rangle$.

In [9]:
program = [
# initialization 
{ "gate": "h", "target": [0] },
{ "gate": "h", "target": [1] },
# oracle 
{ "gate": "cz", "target": [0,1]},
# diffuser
{ "gate": "h", "target": [0] },
{ "gate": "h", "target": [1] },
{ "gate": "z", "target": [0] },
{ "gate": "z", "target": [1] },
{ "gate": "cz", "target": [0,1]},
{ "gate": "h", "target": [0] },
{ "gate": "h", "target": [1] },
]

n = qubit_num(program)

initial_state = get_ground_state(n)

final_state = run_program(initial_state, program)

num_shots = 10000

get_counts(final_state, num_shots)

{'11': 10000}

### 5: $R_x$ and $R_y$ gates

In [10]:
program = [
# rotation abount the x-axis by a pi/2 angle 
{ "gate": "u3", "params": { "theta": np.pi/2, "phi": -np.pi/2, "lambda": np.pi/2 },
 "target": [0] },
# rotation abount the y-axis by a pi/2 angle 
{ "gate": "u3", "params": { "theta": np.pi/2, "phi": 0, "lambda": 0 },
 "target": [1] },
]

n = qubit_num(program)

initial_state = get_ground_state(n)

final_state = run_program(initial_state, program)

num_shots = 10000

get_counts(final_state, num_shots)

{'10': 2480, '00': 2497, '11': 2495, '01': 2528}

## Variational Quantum Algorithms

we will use the last simulator to find the paramaters that give the minimum cost for the circuit:

<img src="circ.jpg">

where the $U$ operation is given by

$$
U(\theta, \phi) = 
\begin{pmatrix}
\cos \left (\frac{\theta}{2}\right) & \sin \left (\frac{\theta}{2}\right) \\ 
e^{\imath \phi} \cos \left (\frac{\theta}{2}\right) & e^{\imath (\phi - \pi)} \cos \left (\frac{\theta}{2}\right)
\end{pmatrix}
$$

which is a special case of $U_3(\theta, \phi, \lambda)$ gate with $\lambda = \pi$. 

The expected value of an operator $\hat{A}$ with eigen value $a_j$

$$
\langle \hat{A}\rangle = \sum_j a_j |\langle \psi|\phi_j \rangle|^2
$$

We measure only in the z-basis, and for one qubit system, we have

$$
 \hat{A} = Z = 
\begin{pmatrix}
1 & 0 \\
0 & -1
\end{pmatrix}
$$

$$
a_j = \{ 1, -1 \}
$$

$$
| \phi_j \rangle = \{ | 0 \rangle, | 1 \rangle \}
$$ 

The cost function is defined as:

$$
f = \langle Z \rangle = |\langle \psi|0 \rangle|^2 - |\langle \psi|0 \rangle|^2 = P_0 - P_1
$$

where $P_0$ is the probability for measuring state $|0\rangle$ and where $P_1$ is the probability for measuring state $|1\rangle$.

In [11]:
def run_variational_circuit(initial_state, program, params):
    # anther form of the run_program function which takes 3 inputs this time.
    psi = initial_state
    for i in range(len(program)):
        gate = program[i]['gate']
        target = program[i]['target']
        
        if gate == 'u3':
            theta1 = params[program[i]['params']['theta']]
            theta2 = params[program[i]['params']['phi']]
            theta3 = params[program[i]['params']['lambda']]
            unitary = u3(theta1, theta2, theta3)
            o = get_parametric_operator(n, unitary, target)
        else:
            o = get_operator(n, gate, target)
        psi = np.dot(psi, o)

    final_state = psi
    return final_state

def objective_function(init_params):
    angles = { "global_1": init_params[0], "global_2": init_params[1], 'global_3': -3.1415 }

    final_state = run_variational_circuit(initial_state, program, angles)

    num_shots = 1000

    counts = get_counts(final_state, num_shots)

    P0 = 0
    P1 = 0

    if '0' in counts:
        P0 = counts['0']/num_shots
    if '1' in counts:
        P1 = counts['1']/num_shots

    cost = P0 - P1
    return cost

In [12]:
program = [{ "gate": "u3", "params": { "theta": "global_1", "phi": "global_2",'lambda': 'global_3'}, "target": [0] }]

n = qubit_num(program)

initial_state = get_ground_state(n)

init_params = [3.1415, 1.5708]

cost = objective_function(init_params)

minimize(objective_function, init_params, method="Powell", tol=1e-6)

   direc: array([[1., 0.],
       [0., 1.]])
     fun: array(-1.)
 message: 'Optimization terminated successfully.'
    nfev: 47
     nit: 1
  status: 0
 success: True
       x: array([3.14148255, 4.1883809 ])

we find that the angles $\theta = 3.13851035$ and $\phi = 4.1883809$ give the minimal cost for the circuit.

## References

<ol>
    <li><a href="https://qiskit.org/documentation/tutorials/circuits/3_summary_of_quantum_operations.html">Qiskit tutorials: Summary of Quantum Operations</a></li>
    <li><a href="https://en.wikipedia.org/wiki/Bell_state">Wikipedia: Bell state</a></li>
    <li><a href="https://en.wikipedia.org/wiki/Greenberger%E2%80%93Horne%E2%80%93Zeilinger_state">Wikipedia: Greenberger–Horne–Zeilinger state
</a></li>
    <li>Lee, Soonchil & Lee, Seong-Joo & Kim, Taegon & Lee, Jae-Seung & Biamonte, Jacob & Perkowski, Marek. (2006). The cost of quantum gate primitives. Journal of Multiple-Valued Logic and Soft Computing. 12. 561-573. </li>
    <li><a href="https://qiskit.org/textbook/ch-algorithms/grover.html">Qiskit Textbook: Grover's Algorithm
</a></li>
    <li><a href="https://en.wikipedia.org/wiki/Expectation_value_(quantum_mechanics)#:~:text=In%20quantum%20mechanics%2C%20the%20expectation,(measurement)%20of%20an%20experiment.">Expectation value (quantum mechanics)
</a></li>
</ol>